In [1]:
#Written by ChatGPT

import heapq
import math

class FibonacciHeapNode:
    def __init__(self, node, key):
        self.node = node
        self.key = key
        self.parent = None
        self.child = None
        self.left = self
        self.right = self
        self.degree = 0
        self.marked = False

class FibonacciHeap:
    def __init__(self):
        self.min_node = None
        self.nodes = {}

    def insert(self, node, key):
        new_node = FibonacciHeapNode(node, key)
        self.nodes[node] = new_node
        if not self.min_node or key < self.min_node.key:
            self.min_node = new_node

    def extract_min(self):
        if self.min_node:
            min_node = self.min_node
            if min_node.child:
                child = min_node.child
                while True:
                    next_child = child.right
                    child.left = min_node
                    child.right = min_node.right
                    min_node.right.left = child
                    min_node.right = child
                    if next_child == min_node.child:
                        break
                    child = next_child
            min_node.left.right = min_node.right
            min_node.right.left = min_node.left
            if min_node == min_node.right:
                self.min_node = None
            else:
                self.min_node = min_node.right
                self.consolidate()
            del self.nodes[min_node.node]
            return min_node.node, min_node.key
        return None, None

    def consolidate(self):
        max_degree = int(2 * (math.log(len(self.nodes)) / math.log(1 + math.sqrt(5))))
        degree_table = [None] * (max_degree + 1)
        current = self.min_node
        unprocessed_nodes = [current]
        while True:
            current = current.right
            if current == self.min_node:
                break
            unprocessed_nodes.append(current)
        for node in unprocessed_nodes:
            degree = node.degree
            while degree_table[degree]:
                other = degree_table[degree]
                if node.key > other.key:
                    node, other = other, node
                self.link(other, node)
                degree_table[degree] = None
                degree += 1
            degree_table[degree] = node
        for entry in degree_table:
            if entry and entry.key < self.min_node.key:
                self.min_node = entry

    def link(self, node1, node2):
        node1.right.left = node1.left
        node1.left.right = node1.right
        node1.parent = node2
        if not node2.child:
            node2.child = node1
            node1.right = node1
            node1.left = node1
        else:
            node1.left = node2.child
            node1.right = node2.child.right
            node2.child.right.left = node1
            node2.child.right = node1
        node2.degree += 1
        node1.marked = False

# def dijkstra(graph, start):
#     distances = {}
#     predecessors = {}
#     heap = FibonacciHeap()
    
#     for node in graph:
#         distances[node] = float('inf')
#     distances[start] = 0
#     heap.insert(start, 0)
#     while heap.min_node:
#         current_node, current_distance = heap.extract_min()
#         print("Current Node:", current_node, "Distance:", current_distance)
#         if current_distance > distances[current_node]:
#             continue
        
#         for neighbor, weight in graph[current_node].items():
#             #print('neighbor', neighbor, 'cur_node', current_node)
#             distance = current_distance + weight
#             #print(distances, neighbor)
#             if distance < distances[neighbor]:
#                 distances[neighbor] = distance
#                 predecessors[neighbor] = current_node
#                 heap.insert(neighbor, distance)
#     #print('Done with Dijkstra')
#     #print('dijkstra', distances)
#     return distances, predecessors

def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    predecessors = {node: None for node in graph}
    distances[start] = 0
    
    priority_queue = [(0, start)]
    
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        
        if current_distance > distances[current_node]:
            continue
        #print(graph)
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node  # Store the predecessor
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances, predecessors

def shortest_path(graph, start, end, predecessors):
    path = []
    while end:
        path.insert(0, end)
        end = predecessors.get(end, None)
    return path

# Example usage:
graph = {
    'A': {'B': 2, 'D': 8, 'G' : 6},
    'B': {'A': 2, 'D': 5, 'E': 6},
    'C': {'E': 9, 'F': 3},
    'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
    'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
    'F': {'D': 2, 'E': 1, 'C': 3},
    'G': {'A': 6, 'E': 4}
}

start_node = 'A'
end_node = 'F'

shortest_distance, predecessors = dijkstra(graph, start_node)
shortest_path_to_D = shortest_path(graph, start_node, end_node, predecessors)

print(f"Shortest distance from {start_node} to {end_node}: {shortest_distance}")
print(f"Shortest path from {start_node} to {end_node}: {' -> '.join(shortest_path_to_D)}")

Shortest distance from A to F: {'A': 0, 'B': 2, 'C': 12, 'D': 7, 'E': 8, 'F': 9, 'G': 6}
Shortest path from A to F: A -> B -> D -> F


In [2]:
import heapq
#import heapq



In [3]:
predecessors

{'A': None, 'B': 'A', 'C': 'F', 'D': 'B', 'E': 'B', 'F': 'D', 'G': 'A'}

In [4]:
def create_directed_graph(graph, shortest_distance):
    '''Creates a directed graph as specified on page 19 or 245 of AN19.'''
    directed = {}
    for u in graph: #first vertex
        if u not in directed:
            directed[u] = {}
        for v in graph[u]: #second vertex
            directed[u][v] = graph[u][v] - (shortest_distance[v] - shortest_distance[u])
            #directed[v][u] = graph[v][u] - (shortest_distance[u] - shortest_distance[v])
    return directed

In [5]:
d = create_directed_graph(graph, shortest_distance)

In [6]:
d

{'A': {'B': 0, 'D': 1, 'G': 0},
 'B': {'A': 4, 'D': 0, 'E': 0},
 'C': {'E': 13, 'F': 6},
 'D': {'A': 15, 'B': 10, 'E': 2, 'F': 0},
 'E': {'B': 12, 'C': 5, 'D': 4, 'F': 0, 'G': 6},
 'F': {'D': 4, 'E': 2, 'C': 0},
 'G': {'A': 12, 'E': 2}}

In [7]:
dijkstra(d, 'E')

({'A': 16, 'B': 12, 'C': 0, 'D': 4, 'E': 0, 'F': 0, 'G': 6},
 {'A': 'B', 'B': 'E', 'C': 'F', 'D': 'E', 'E': None, 'F': 'E', 'G': 'E'})

In [8]:
dijkstra(graph, 'A')

({'A': 0, 'B': 2, 'C': 12, 'D': 7, 'E': 8, 'F': 9, 'G': 6},
 {'A': None, 'B': 'A', 'C': 'F', 'D': 'B', 'E': 'B', 'F': 'D', 'G': 'A'})

In [9]:
def is_undirected(graph):
    # Create a set to store checked edges
    checked_edges = set()

    # Iterate through each node in the graph
    for node, neighbors in graph.items():
        for neighbor, length in neighbors.items():
            # Check if the reverse edge exists and has the same length
            if (neighbor, node) in checked_edges:
                if graph[neighbor][node] != length:
                    print(f'neigbhor = {neighbor} and node = {node}')
                    return False
            else:
                checked_edges.add((node, neighbor))

    return True

In [10]:
def create_petal(graph, x, t, r, directed = None): #Edit this to take in the directed graph as a parameter Y that is computed separately
    assert x in graph
    assert t in graph
    shortest_distance, predecessors = dijkstra(graph, x)
    if directed == None:
        directed = create_directed_graph(graph, shortest_distance)
        
    
    shortest_path_to_t = shortest_path(graph, x, t, predecessors)
    if x != t and len(shortest_path_to_t) == 1:
        print(x, t, shortest_path_to_t)
    shortest_distance_directed, predecessors_directed = dijkstra(directed, t)
    #shortest_path_to_x = shortest_path(directed, t, x, predecessors_directed)
    #print(shortest_path_to_x)
    t_dist = shortest_distance[t]
    #print(t_dist)
    #print(shortest_path_to_t)
    r_prime = x
    shortest_path_rprime = []
    r_prime_idx = -1
    for i in range(len(shortest_path_to_t)):
        v = shortest_path_to_t[i]
        d = t_dist - shortest_distance[v]
        #shortest_path_rprime.append(v)
        #print(d)
        if d < r:
            r_prime = v
            r_prime_idx = i
            break
    #print(r_prime)
    
    for i in range(r_prime_idx, len(shortest_path_to_t)):
        shortest_path_rprime.append(shortest_path_to_t[i])
    
    #shortest_path_rprime = shortest_path(directed, t, r_prime, predecessors_directed)
    #print(shortest_path_rprime)
    
    petal = {}
    for v in shortest_distance_directed:
        if shortest_distance_directed[v] < r/2:
            petal[v] = {}
    for v in shortest_path_rprime:
        petal[v] = {}
        
    for u in petal:
        for v in graph[u]:
            if v in petal:
                petal[u][v] = graph[u][v]
    #print(x, r_prime, shortest_path_to_t, t)
    #print(f'length in create_petal: {t_dist}')
    #print(r_prime, shortest_path_to_t)
    #assert r_prime in shortest_path_to_t
    if not is_undirected(petal):
        print(petal)
        raise ValueError('Petal not undirected')
    return petal, r_prime, shortest_path_to_t
    #for i in range(len(shortest_path_rprime - 1)):
        
    
#     shortest_distance, predecessors = dijkstra(directed, t)
#     _, predecessors = dijkstra(graph, t)
#     shortest_path_to_x = shortest_path(graph, t, x, predecessors) #shortest_path(directed, t, x, predecessors) #NEED WEIGHTS TO BE HALF OF WHAT THEY WERE IN G! P_t, r_primeshould be a path in G
#     r_prime = t
#     shortest_distance_directed, predecessors_directed = dijkstra(directed, t)
#     for i in range(1, len(shortest_path_to_x)):
#         v = shortest_path_to_x[i]
#         shortest_distance_directed[v] /= 2
#         if shortest_distance[v] > r:
#             r_prime = shortest_path_to_x[i - 1]
#             break
#         break
#     petal = {}
#     #print(directed)
#     print(shortest_distance_directed)
#     for v in shortest_distance_directed:
#         if shortest_distance_directed[v] < r/2:
#             petal[v] = {}
    
#     for u in directed:
#         for v in directed[u]:
#             if u in petal and v in petal:
#                 petal[u][v] = directed[u][v]
    
#     assert r_prime in petal
    
#     return petal, r_prime

In [75]:
graph = {'0': {'1': 37.0}, '1': {'0': 37.0, '6': 92.0}, '2': {'3': 5.0}, '3': {'2': 5.0, '4': 30.0}, '4': {'3': 30.0, '6': 20.0}, '5': {'6': 4.0}, '6': {'5': 4.0, '4': 20.0, '1': 92.0}}

In [80]:
hierarchical_petal_decomposition(graph, '1', '5')

{'1': {'0': 37.0, '6': 92.0},
 '0': {'1': 37.0},
 '6': {'1': 92.0, '4': 5.0, '5': 4.0},
 '4': {'6': 5.0, '3': 30.0},
 '5': {'6': 4.0},
 '3': {'2': 1.25, '4': 30.0},
 '2': {'3': 1.25}}

In [11]:
#create_petal(graph, 'A', 'C', 12 - 5*12/8)

({'C': {'F': 3}, 'F': {'C': 3}}, 'F', ['A', 'B', 'D', 'F', 'C'])

In [12]:
#create_petal(graph, 'A', 'C', 12 - 5*12/8)

({'C': {'F': 3}, 'F': {'C': 3}}, 'F', ['A', 'B', 'D', 'F', 'C'])

In [13]:
graph

{'A': {'B': 2, 'D': 8, 'G': 6},
 'B': {'A': 2, 'D': 5, 'E': 6},
 'C': {'E': 9, 'F': 3},
 'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
 'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'A': 6, 'E': 4}}

In [14]:
def maintain_graph_difference(G, H):
    result = {}

    for u, neighbors in G.items():
        if u not in H:
            result[u] = {}
        
        for v in neighbors:
            if u not in H and v not in H:
                if u not in result:
                    result[u] = {}
                result[u][v] = G[u][v]

    return result

In [15]:
graph

{'A': {'B': 2, 'D': 8, 'G': 6},
 'B': {'A': 2, 'D': 5, 'E': 6},
 'C': {'E': 9, 'F': 3},
 'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
 'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'A': 6, 'E': 4}}

In [16]:
maintain_graph_difference(graph, ['A', 'B'])

{'C': {'E': 9, 'F': 3},
 'D': {'E': 3, 'F': 2},
 'E': {'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'E': 4}}

In [174]:
def petal_decomposition(graph, x, t):
    assert x in graph
    if t not in graph:
        raise ValueError(f't = {t} not in graph vertices = {list(graph.keys())}')
    shortest_distance, predecessors = dijkstra(graph, x)
    r = max(shortest_distance.values())
    #print('d', shortest_distance)
    Ys = [graph]
    Xs = []
    ys = []
    xs = [x]
    ts = []
    j = 1
    if shortest_distance[t] > 5*r/8:
        
        X_1, x_1, path = create_petal(graph, x, t, shortest_distance[t] - 5*r/8, graph)
        #print('special petal', X_1, x_1, t, path)
        #print('HI', X_1, x_1, t)
        #Ys.append({key : value for key, value in Ys[0].items() if key not in X_1})
        Ys.append(maintain_graph_difference(Ys[0], X_1))
        Xs.append(X_1)
        xs.append(x_1)
        
        y1 = x
        min_distance = r
        #if x_1 != t:
        #path = shortest_path(graph, x, t, predecessors)
        #print('path', path)
        for i in range(len(path)):
            if path[i] == x_1:
                y1 = path[i-1]
                break

        j = 2
        ts = [y1, t]
        ys = [y1]
        if y1 in X_1 and x_1 != t:
            raise ValueError(f'y1 = {y1} should not be in X1 = {X_1}')
        
    else:
        ts = [t] 
    while True:
        #shortest_distance, predecessors = dijkstra(graph, x)
        ball = {key : value for key, value in shortest_distance.items() if shortest_distance[key] < 3*r/4}
        leftover = [v for v in Ys[-1] if v not in ball] #maintain_graph_difference(Ys[-1], ball)
        if len(leftover) == 0:
            break
        else:
            #print('leftover', leftover, '\n')
            ts.append(leftover[0])
            X_j, x_j, path = create_petal(graph, x, ts[-1], r/8, Ys[-1])
            #print(f'X_j = {X_j}, graph = {graph}, x = {x}, t_j = {ts[-1]}, r = {r/8}, Ys[-1] = {Ys[-1]}')
            if x == ts[-1] and x_j != x:
                print('path of length 1:', x, x_j)
            if x_j not in X_j:
                print(f'x_j = {x_j} not in X_j = {X_j}')
                print(f'graph = {graph}, x = {x}, t_j = {ts[-1]}, r = {r/8}, Ys[-1] = {Ys[-1]}')
                raise ValueError
            assert ts[-1] in X_j
            Xs.append(X_j)
            xs.append(x_j)
            Ys.append(maintain_graph_difference(Ys[-1], X_j))
            xj_idx = -1
            if x_j not in path:
                print(f'x_j {x_j} not found in path {path} of length {shortest_distance[ts[-1]]}')
            for i in range(len(path)):
                if path[i] == x_j:
                    xj_idx = i
                # else:
                #     print(f'x_j {x_j} not found in path {path}')
                if xj_idx > -1 and i < len(path) - 1:
                    #print('g', graph, path[i-1], path[i], path, i)
                    graph[path[i]][path[i+1]] /= 2
                    graph[path[i+1]][path[i]] /= 2
                    shortest_distance[path[i]] = (shortest_distance[path[i]] + shortest_distance[path[xj_idx]])/2
            if x_j == x:
                print('hello')
                ys.append(path[0])
            else:
                ys.append(path[xj_idx - 1])
                #print('path', path, 'x_j', path[xj_idx], 'y_j', path[xj_idx - 1])
            j += 1
            if ys[-1] == xs[-1]:
                print(path)
                #raise ValueError('x = y')
            if ys[-1] not in graph[xs[-1]]:
                raise ValueError(f'{xs[-1]}, {ys[-1]} not an edge in graph')
            #print('ys', ys)
        #print('Completed while')
    s = j - 1
    X0 = Ys[s]
    Xs = [X0] + Xs
    return Xs, ys, xs, ts
        

In [175]:
petal_decomposition(graph, '2', '3')

([{'0': {}, '2': {'6': 1.0, '4': 33.0}, '4': {'2': 33.0}, '6': {'2': 1.0}},
  {'1': {'3': 3.0}, '3': {'1': 3.0}},
  {'5': {}}],
 ['2', '0'],
 ['2', '1', '5'],
 ['2', '3', '5'])

In [176]:
def make_undirected(graph):
    undirected_graph = {}
    for u in graph:
        for v, length in graph[u].items():
            # Add edge from u to v
            if u not in undirected_graph:
                undirected_graph[u] = {}
            undirected_graph[u][v] = length
            
            # Add edge from v to u
            if v not in undirected_graph:
                undirected_graph[v] = {}
            undirected_graph[v][u] = length
    
    return undirected_graph

In [177]:
def hierarchical_petal_decomposition(graph, x_0, t):
    
    if len(list(graph.keys())) == 1:
        return graph
    if is_tree(graph):
        return graph
    Xs, ys, xs, ts = petal_decomposition(graph, x_0, t)
    #print(f'Xs = {Xs}, ys = {ys}, xs = {xs}, ts = {ts}')
    #print(Xs[0], xs[0])
    Ts = []
    for j in range(len(Xs)):
        if xs[j] not in Xs[j]:
            print(f'xj = {xs[j]} not in {Xs[j]}')
        if ts[j] not in Xs[j]:
            print(f'tj = {xs[j]} not in {Xs[j]}')
        Ts.append(hierarchical_petal_decomposition(Xs[j], xs[j], ts[j]))
    #print(Ts)
    T = {}
    for tree in Ts:
        if not is_undirected(tree):
            print(tree)
            raise ValueError('tree is not undirected')
        for u in tree:
            T[u] = tree[u]
    #print(xs, ys)
    if len(xs) > 1:
        for i in range(1, len(xs)):
            x = xs[i]
            y = ys[i-1]
            T[x][y] = graph[x][y]
            T[y][x] = graph[y][x]
    #print('T', T)
    
    #Make T undirected
    #T = make_undirected(T)
    # if not is_connected(T):
    #     print(T, xs, ys)
    #     raise ValueError('t is not connected')
    # if not is_undirected(T):
    #     print(T)
    #     raise ValueError('t is not undirected')
    
    return make_undirected(T)
        

In [178]:
tree = hierarchical_petal_decomposition(graph, 'A', 'C')

AssertionError: 

In [180]:
tree

{'0': {'2': 85.0, '5': 8.0},
 '1': {'4': 18.0},
 '2': {'0': 85.0, '6': 50.0, '4': 96.0},
 '3': {'4': 93.0},
 '4': {'1': 18.0, '3': 93.0, '2': 96.0},
 '5': {'0': 8.0},
 '6': {'2': 50.0}}

### Tree

In [95]:
import numpy as np
from tqdm import tqdm
def calculate_stretch(graph, x_0, t):
    T = hierarchical_petal_decomposition(graph, x_0, t)
    assert is_undirected(T)
    T = make_undirected(T)
    #print(T)
    if len(T) != len(graph):
        raise ValueError(f'Not a spanning tree. T has {len(t)} vertices while the original graph has {len(graph)} vertices')
    m = len(graph)
    n = 0
    #t_length = 0
    #graph_length = 0
    stretch = 0
    graph_keys_list = list(graph.keys())
    for i in tqdm(range(len(graph_keys_list))):
        u = graph_keys_list[i]
        for v in graph[u]:
            assert u in T and v in T
            n += 1
            distances, _ = dijkstra(T, u)
            if distances[v] == float('inf'):
                print('INF', u, v, T)
                print(is_undirected(T))
                print(is_connected(T))
                raise ValueError('Distance should be finite')
            stretch += distances[v]/graph[u][v]
    print(f'fraction of edges: {(n/2)/(m*(m-1)/2) }')
    n /= 2
    bound = m * np.log(n) * np.log(np.log(n))
    #print(f'stretch = {stretch}')
    return stretch/bound

In [96]:
import random

def generate_graph(m, p):
    if m <= 1:
        # Special case: a single vertex graph is always connected.
        return {'0': {}}

    # Create a list of vertices as strings from '0' to 'm-1'.
    vertices = [str(i) for i in range(m)]

    # Create a dictionary to represent the graph with edge lengths.
    graph = {v: {} for v in vertices}

    # Create a list of edges with associated probabilities.
    edges = []

    for i in range(m):
        for j in range(i + 1, m):
            if random.random() < p:
                edges.append((vertices[i], vertices[j]))

    # Create a minimum spanning tree using Prim's algorithm.
    random.shuffle(vertices)
    visited = set([vertices[0]])
    while len(visited) < m:
        current_vertex = random.choice(list(visited))
        remaining_vertices = [v for v in vertices if v not in visited]
        if not remaining_vertices:
            break
        next_vertex = random.choice(remaining_vertices)
        visited.add(next_vertex)
        edge = (current_vertex, next_vertex)
        edges.append(edge)

    # Shuffle the list of edges to randomize the order.
    random.shuffle(edges)

    # Add the edges to the graph.
    for (start_vertex, end_vertex) in edges:
        edge_length = float(random.randint(1.0, 100.0))  # Modify this range as needed
        graph[start_vertex][end_vertex] = edge_length
        graph[end_vertex][start_vertex] = edge_length

    return graph

# # Example usage:
# m = 5
# p = 0.5  # Probability of an edge between any pair of vertices
# connected_graph = generate_connected_graph_with_probability(m, p)
# print(connected_graph)

In [97]:
def is_connected(graph):
    if not graph:
        # An empty graph is considered connected
        return True

    def dfs(node, visited):
        visited[node] = True
        for neighbor in graph[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited)

    # Initialize a dictionary to keep track of visited nodes
    visited = {node: False for node in graph}

    # Choose any node as the starting point
    start_node = next(iter(graph))
    dfs(start_node, visited)

    # Check if all nodes have been visited
    return all(visited[node] for node in graph)

# Example usage:
graph = {
    'A': {'B': 2, 'D': 8, 'G' : 6},
    'B': {'A': 2, 'D': 5, 'E': 6},
    'C': {'E': 9, 'F': 3},
    'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
    'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
    'F': {'D': 2, 'E': 1, 'C': 3},
    'G': {'A': 6, 'E': 4}
}

connected = is_connected(graph)
if connected:
    print("The graph is connected.")
else:
    print("The graph is not connected.")

The graph is connected.


In [170]:
def furthest_point(graph, x_0):
    distances, _ = dijkstra(graph, x_0)
    #print(distances)
    max_dist = max(distances.values())
    for v in distances:
        if distances[v] == max_dist:
            return v

In [171]:
furthest_point(graph, '0')

'6'

In [172]:
import random

# Create a graph represented as a dictionary

def test_lsst(m, p):

#     graph = {}
#     vertices = [str(i) for i in range(m)]

#     # Create edges with random lengths
#     for i in range(m):
#         for j in range(i + 1, m):
#             if np.random.uniform() < p:
#                 edge_length = np.random.randint(1, 100)  # Random edge length between 1 and 10
#                 if vertices[i] not in graph:
#                     graph[vertices[i]] = {}
#                 if vertices[j] not in graph:
#                     graph[vertices[j]] = {}
#                 graph[vertices[i]][vertices[j]] = edge_length
#                 graph[vertices[j]][vertices[i]] = edge_length  # Undirected, so add the reverse edge
#             else:
#                 print('oops')

    graph = generate_graph(m, p)
    assert is_connected(graph)
            
    v1 = '0' #str(np.random.randint(0, m))
    v2 = furthest_point(graph, v1) #str(np.random.randint(0, m))
    
    #print(graph, v1, v2)
    return calculate_stretch(graph, v1, v2)

# Example: Accessing the length of the edge between vertex '0' and '1'
#edge_length_01 = graph['0']['1']

In [181]:
import time
n = 200
for i in range(n):
    for num_vertices in [7]: #, 1000, 5000]:
        print(f'Starting trial {i + 1} of {n}')
        p = np.log(num_vertices)/num_vertices
        print(test_lsst(num_vertices, 0.1))#np.log(1000)/1000))
            
            
        #print(f'LSST on graph with {num_vertices} vertices')
        # start = time.time()
        # print(test_lsst(num_vertices, 1))
        # end = time.time()
        #print(f'Took {end - start} seconds')

Starting trial 1 of 200


100%|██████████| 7/7 [00:00<?, ?it/s]


fraction of edges: 0.38095238095238093
1.5632089450997693
Starting trial 2 of 200


100%|██████████| 7/7 [00:00<00:00, 61941.20it/s]


fraction of edges: 0.3333333333333333
1.8371261591999717
Starting trial 3 of 200


...

100%|██████████| 7/7 [00:00<?, ?it/s]


fraction of edges: 0.3333333333333333
1.3911746904759679
Starting trial 60 of 200


100%|██████████| 7/7 [00:00<00:00, 11096.04it/s]

fraction of edges: 0.38095238095238093
1.9371132837454081
Starting trial 61 of 200
0 1 ['1']
x_j = 0 not in X_j = {'1': {}}
graph = {'0': {'4': 77.0}, '1': {}, '4': {'0': 77.0, '6': 10.0}, '6': {'4': 10.0}}, x = 0, t_j = 1, r = inf, Ys[-1] = {'0': {'4': 77.0}, '1': {}, '4': {'0': 77.0, '6': 10.0}, '6': {'4': 10.0}}


ValueError: 

### Debugging

In [101]:
graph = {'0': {'5': 36.0, '1': 8.0}, '1': {'3': 3.0, '0': 8.0, '2': 83.0}, '2': {'6': 1.0, '4': 33.0, '1': 83.0}, '3': {'4': 67.0, '1': 3.0, '6': 98.0}, '4': {'3': 67.0, '2': 33.0}, '5': {'0': 36.0}, '6': {'2': 1.0, '3': 98.0}}

In [130]:
petal_decomposition(graph, '2', '3')

special petal {'1': {'3': 3.0}, '3': {'1': 3.0}} 1 3 ['2', '1', '3']


([{'0': {}, '2': {'6': 1.0, '4': 33.0}, '4': {'2': 33.0}, '6': {'2': 1.0}},
  {'1': {'3': 3.0}, '3': {'1': 3.0}},
  {'5': {}}],
 ['2', '0'],
 ['2', '1', '5'],
 ['2', '3', '5'])

In [127]:
petal_decomposition({'0': {}, '2': {'6': 1.0, '4': 33.0}, '4': {'2': 33.0}, '6': {'2': 1.0}}, '2', '2')

2 0 ['0']
x_j = 2 not in X_j = {'0': {}}
graph = {'0': {}, '2': {'6': 1.0, '4': 33.0}, '4': {'2': 33.0}, '6': {'2': 1.0}}, x = 2, t_j = 0, r = inf, Ys[-1] = {'0': {}, '2': {'6': 1.0, '4': 33.0}, '4': {'2': 33.0}, '6': {'2': 1.0}}


ValueError: 

In [104]:
def is_tree(graph):
    num_vertices = len(graph)
    num_edges = sum(len(adj_list) for adj_list in graph.values()) // 2  # Divide by 2 to account for double-counting edges

    return num_edges == num_vertices - 1

# Example usage

if is_tree(graph):
    print("The graph is a tree.")
else:
    print("The graph is not a tree.")


The graph is not a tree.


In [105]:
graph

{'0': {'5': 36.0, '1': 8.0},
 '1': {'3': 3.0, '0': 8.0, '2': 83.0},
 '2': {'6': 1.0, '4': 33.0, '1': 83.0},
 '3': {'4': 67.0, '1': 3.0, '6': 98.0},
 '4': {'3': 67.0, '2': 33.0},
 '5': {'0': 36.0},
 '6': {'2': 1.0, '3': 98.0}}